In [25]:
import requests
import toml
import os
import subprocess
from packaging import version

In [26]:
url = "https://raw.githubusercontent.com/toftul/latexdiff-bot/main/pyproject.toml"
pyproject_content_gh = requests.get(url).text
pyproject_content_gh_parsed = toml.loads(pyproject_content_gh)
gh_version = pyproject_content_gh_parsed['tool']['poetry']['version']
local_version = '0.1.0'

In [27]:
print(gh_version)

0.1.1


In [28]:
print(local_version)

0.1.0


In [29]:
version.parse(gh_version) > version.parse(local_version)

True

In [30]:
def check_pull_restart(working_dir, gh_raw_url_toml, service_name, local_toml_filename='pyproject.toml'):
    default_dir = os.getcwd()
    
    pyproject_content_gh = requests.get(gh_raw_url_toml).text
    pyproject_content_gh_parsed = toml.loads(pyproject_content_gh)
    gh_version = pyproject_content_gh_parsed['tool']['poetry']['version']
    
    pyproject_content_local_parsed = toml.load(os.path.join(working_dir, local_toml_filename))
    local_version = pyproject_content_local_parsed['tool']['poetry']['version']
    
    if version.parse(gh_version) > version(local_version):
        # change dir
        os.chdir(working_dir)
        # pull 
        subprocess.run(
            'git pull',
            shell=True
        )
        # install new dependencies if any
        subprocess.run(
            'poetry install',
            shell=True
        )
        # restart service
        subprocess.run(
            f'systemctl --user restart {service_name}',
            shell=True
        )
        os.chdir(default_dir)
        
        return True
    else:
        return False
    

In [31]:
bots = {
    "latexdiffbot": {  # https://github.com/toftul/latexdiff-bot
        "working_dir": "/home/ivan/bots/latexdiffbot",
        "gh_raw_url_toml": "https://raw.githubusercontent.com/toftul/latexdiff-bot/main/pyproject.toml",
        "service_name": "latexdiffbot"
    },
    "tg-howwasyourdaybot": {  # https://github.com/toftul/tg-howwasyourday
        "working_dir": "/home/ivan/bots/tg-howwasyourday",
        "gh_raw_url_toml": "https://raw.githubusercontent.com/toftul/tg-howwasyourday/main/bot.py?token=GHSAT0AAAAAACDDKS5A5AX6VADTNT3IWK3QZDSCRQA",
        "service_name": "howwasyourdaybot"
    }
}

In [33]:
requests.get("https://raw.githubusercontent.com/toftul/tg-howwasyourday/main/bot.py?token=GHSAT0AAAAAACDDKS5A5AX6VADTNT3IWK3QZDSCRQA")

<Response [404]>

In [32]:
for bot in bots:
    check_pull_restart(
        working_dir=bots[bot]["working_dir"], 
        gh_raw_url_toml=bots[bot]["gh_raw_url_toml"], 
        service_name=bots[bot]["service_name"], 
        local_toml_filename='pyproject.toml'
    )

FileNotFoundError: [Errno 2] No such file or directory: '/home/ivan/bots/latexdiffbot/pyproject.toml'